---------------------------------------------------------

# PhAst Algorithm (Part 1 of 4): 

# Imaging the Asteroid and Data Pre-processing 

## By Arushi Nath

----------------------------------------------

#### This First Notebook will walk you through the steps of pre-processsing your observation of Didymos before analysis. It uses libaries such as Astropy and Astroquery to extract metadata from your images and use it to query for all known stars in the field of view (later used when finding comparison stars). In addition, it will query for the celestial coordinates and magnitude of Didymos across the full duration of an observation.

#### Note: If you have multiple observation nights of Didymos, you will need to re-run the code seperately for each one.

----------------------------------------------

### 1. Preparing Coding Environment by Installing Libraries

In [ ]:
### Importing Required Libraries ###

# Basic Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

# Used to help with analysis
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u 
import astropy.coordinates as coord
import astropy.coordinates as coord
import astropy.units as u
from astropy.wcs import WCS
from astropy.visualization import simple_norm
from astropy.utils.data import get_pkg_data_filename

# Used for Trying out Different Aperture Sizes for Differential Photometry
import photutils 
from photutils.aperture import CircularAperture, CircularAnnulus
from photutils.datasets import make_100gaussians_image
from photutils.aperture import ApertureStats
from photutils.aperture import aperture_photometry

# Used to Query Open Datasets
from astroquery.mpc import MPC
from astroquery.vizier import Vizier 
from astroquery.gaia import Gaia
import urllib as url
import requests

# Curve Fitting and Normalization
from scipy import stats
from scipy.optimize import curve_fit
from sklearn import preprocessing

### 2. Getting Image and Essential Meta Data from FITS File

In [ ]:
### Querying for FITS files of all Images in Observation ###

images = []
headers = []
n = 266
while n<= 344:
    i = r'D:\Desktop\i-Telescope Observations\Didymos\LCO\17 November 2022\lco_data-20221118-158\lco_data-20221118-158\coj1m011-fa12-20221117-0',str(n),'-e91.fits'
   
    a = "".join(i)
    img = fits.open(a)
    file = img[0]
    images.append(fits.getdata(a))
    headers.append(file.header)
    
    n+=1

In [ ]:
### Showing All Images ###
for i in range(len(images)):
    plt.figure(figsize=(15, 15))
    image_data = images[i]
    plt.imshow(image_data, cmap='gray')
    plt.show()

In [ ]:
### Extracting Usefull Information from FITS File ###
# *Note: Different Telescopes give Different Names for each Data points. It may be Nessesary to Change Query Parameters

#Image Information

DATE = []
for i in range(len(headers)):    
    DATE.append(headers[i]['DATE-OBS'])

BINNING = headers[1]['CCDSUM'][0]


#Camera

NAXIS1 = headers[1]['NAXIS1']
NAXIS2 = headers[1]['NAXIS2']

Pixel_Scale = headers[1]['PIXSCALE']
CCDXPIXE = headers[1]['CCDXPIXE']
CCDXPIXE = CCDXPIXE*1000

FOV =  (int(NAXIS1) * (Pixel_Scale)/60)*2

#OBJECT

RA = headers[1]['RA']
DEC = headers[1]['DEC']


# Printing out all Relevant MetaData

print('Pixel Size (meters) : ', CCDXPIXE)
print('DATE-OBS: ', DATE[1])
print('Right Ascension (hh:mm:ss): ', RA)
print('Declination (degrees: arcmin: arcsec): ', DEC)
print('CCD Length (pixels): ', NAXIS1)
print('CCD Width (pixels): ', NAXIS2)
print('Pixel_Scale (arcsec/pixel): ', Pixel_Scale)
print('BINNING: ', BINNING)
print("Field of View (arcmin): ", FOV)


### 3. Scaling Images to Show Brighter and Dimmer Objects in the Image 

In [ ]:
#### Scaling Image to Clearly View all Objects in Image ###

def show(im):
    plt.imshow(im, cmap='gray', vmin=image_data.mean()-0.2*image_data.std(), vmax=image_data.mean()+0.2*image_data.std())

image_data = images[10]

plt.figure(figsize=(15, 15))
show(image_data)
plt.colorbar()


In [ ]:
### To clip hot pixels / extraneous values / noise ###
def scale(im):
    VMIN = image_data.mean()-0.2*image_data.std() # Sets pixels below VMIN to VMIN
    VMAX = image_data.mean()+0.2*image_data.std() # Sets pixels above VMAX to VMAX
    
    return np.clip(im, VMIN, VMAX)

### 4. Creating a Function to Convert Right Ascension and Declination Coordinates from HH:MM:SS to Degrees

In [ ]:
### Creating a Function to Convert Right Ascention (h : m : s) to Degrees ###

def RAtoDeg(ra):
    if ':' in ra:
        sep = ':'
        
    if ' ' in ra:
        sep = ' '
        
    if '-' in ra:
        sep = '-'
        
    h,m,s = ra.split(sep)
    h,m,s = float(h), float(m), float(s)
   
    
    deg = (h*15) + (m/4) + (s/240)
    print(deg)
    return deg


### Creating a Function to Convert Declination (deg : arc-min : arc-sec) to Degrees ###

def DECtoDeg(dec):
    if ':' in dec:
        sep = ':'
        
    if ' ' in dec:
        sep = ' '
    
    dem = dec[1:]
    if '-' in dem:
        sep = '-'
        
    d,d2,d3 = dec.split(sep)
    
    if d[0] == '-':
        d = float(d)
        d2 = float(d2)
        d3 = float(d3)

    
        deg = d - (d2/60) - (d3/3600)
    
 
    else:    
        d = float(d)
        d2 = float(d2)
        d3 = float(d3)

    
        deg = d + (d2/60) + (d3/3600)
    print(deg)
    return deg

In [ ]:
### Converting Image RA and DEC to degrees ###
RA_deg = RAtoDeg(RA)
DEC_deg = DECtoDeg(DEC)

### 5. Querying GAIA-EDR3 Star Catalog for Celestial Coordinates of all Know Stars in the Image

In [ ]:
###Querying the GAIA Database for Known Stars in the Image ###

Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source"
Gaia.ROW_LIMIT = -1

coord = SkyCoord(ra=RA_deg, dec=DEC_deg, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(FOV/2 +0.1, u.arcmin)
height = u.Quantity(FOV/2+0.1, u.arcmin)

r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

rad = np.array(r['ra'])
ded = np.array(r['dec'])
rmag = np.array(r['phot_g_mean_mag'])

In [ ]:
### Limiting to only Stars whose Magnitude is Brighter than 21 ###
rad_f = []
ded_f = []
rmag_f = []
mag_max = 21
for i in range(len(rmag)):
    if rmag[i] < mag_max:
        rad_f.append(rad[i])
        ded_f.append(ded[i])
        rmag_f.append(rmag[i])

In [ ]:
### Converting all Star RA and DEC Coordinates to Pixel Coordinates ###
x_y_cords = []
star_x = []
star_y = []

wcs = WCS(headers[0])

for ra1,dec1 in zip(rad_f,ded_f):
    
    c = SkyCoord(ra1, dec1, frame='icrs', unit='deg')
    x, y = wcs.world_to_pixel(c)
    
    star_x.append(x)
    star_y.append(y)
    x = 0
    y = 0

In [ ]:
### plot all pixel cordinates on top of image ###
plt.figure(figsize=(15, 15))

plt.imshow(image_data, cmap='gray', vmin=image_data.mean()-0.2*image_data.std(), vmax=image_data.mean()+0.2*image_data.std())
plt.scatter(star_x,star_y,10)

plt.show()

### 6. Querying the JPL Horizons System for Didymos Celestial Coordinates and Magnitude across the full Duration of the Observation

In [ ]:
### Setting Querying Parameters for Didymos ###
mag_limit = 18
def known_asteroid_finder(date, time, fov1, fov2, mpc, ra, dec): #Date: year-month-day   Time: hh:mm:ss   FOV1: arcmin_start to end ra   FOV2: arcmin_start to end dec   MPC code: observatory minor planet center code

    sep = ''
    mpc_mod = sep.join("&mpc-code=" + mpc)
    date_mod = sep.join("&obs-time=" + date)
    time_mod = sep.join("_" + time)
    fov1_mod = sep.join("&fov-ra-center=" + ra)
    fov2_mod = sep.join("&fov-dec-center=" + dec)
    ra_mod = sep.join("&fov-ra-hwidth=" + fov2)
    dec_mod = sep.join("&fov-dec-hwidth=" + fov1)
    mag_mod = sep.join("&vmag-lim=" + str(mag_limit))
   
    start = 'https://ssd-api.jpl.nasa.gov/sb_ident.api?sb-kind=a&mag-required=true&two-pass=true&suppress-first-pass=true&req-elem=false'
    link = start, mpc_mod, date_mod, time_mod, fov1_mod, fov2_mod, ra_mod, dec_mod, mag_mod
    link = sep.join(link)
    return link

In [ ]:
### Creating a Function to Convert RA and Dec into Required Format ###
def convert(ra,dec):
    if '+' in dec:
        dec = dec.replace("+", "")
    if '-' in dec:
        dec = dec.replace("+", "m")
    if '/' in dec:
        dec = dec.replace("/", "")

    ra1 = ra.split(':')
    dec1 = dec.split(':')
    sep = '-'
    ra = sep.join(ra1)
    dec = sep.join(dec1)
    return ra,dec

In [ ]:
# Changing the Format of Meta Data to match the NASA Horizons Database Input Format ###
ra,dec = convert(RA, DEC)
fov1 = str((FOV/60)/2)
fov2 = str((FOV/60)/2)
mpc = 'E10'
ra = ra[0:10]
dec = dec[0:10]

In [ ]:
### Querying for Didymos in each Image ###
asteroids = []
ast_catalog_mag = []
for s in tqdm(range(len(headers))):
    date, time = DATE[s].split('T')
    if '.' in time:
        time = time.split('.')
        time = time[0]
        
    asteroid_query = known_asteroid_finder(date, time, fov1, fov2, mpc, ra, dec)
    
    response_API = requests.get(asteroid_query)
    data = response_API.text

    data = data.split('[')
    
    for i in (range(len(data)-3)):
        ast = data[i + 3]
        ast = ast.split(',')
        
        a = data[i+3].split('"')
        if a[1] == "65803 Didymos (1996 GT)":
            Asteroid = a
        
            x,y = convert(a[3],a[5])
        ast_catalog_mag.append(a[14])
    asteroids.append([x,y])

In [ ]:
### Converting RA and Dec of Didymos to Format recognized by my Algorithm ###
def convert(ra,dec):
    if '+' in dec:
        dec = dec.replace("+", "")
    if '-' in dec:
        dec = dec.replace("-", "m")
  

    ra1 = ra.split(' ')
    dec1 = dec.split(' ')
    sep = '-'
    ra = sep.join(ra1)
    dec = sep.join(dec1)
    return ra,dec

In [ ]:
X,Y = convert(x,y)

In [ ]:
### Finding Didymos RA, DEC, and magnitude from Data Queried ###
Didymos_ra = []
Didymos_dec = []

for a,b in asteroids:
    x1 = []
    y1 = []
    for i in a:
        try:
            i = int(i)
            x1.append(str(i))
            
        except:
            pass
        
    x1.insert(2, ' ')
    x1.insert(5, ' ')
    if len(a)>8:
        x1.insert(8, '.')
    
    x1 = ''.join(str(e) for e in x1)
    
    for i2 in b:
        try:
            y1.append(int(i2))
        except:
            pass
        
    y1.insert(2, ' ')
    y1.insert(5, ' ')
    if len(b)>8:
        y1.insert(8, '.')
    
    y1 = ''.join(str(e) for e in y1)
    
    Didymos_ra.append(x1)
    Didymos_dec.append(y1)

In [ ]:
Didymos_ra1 = []
Didymos_dec1 = []
for i in range(len(Didymos_ra)):
    Didymos_ra1.append(RAtoDeg(Didymos_ra[i]))
    Didymos_dec1.append(DECtoDeg(Didymos_dec[i]))

In [ ]:
### Converting RA and DEC of Known Asteroids to Pixel Cordinates ###
x_y_didymos = []
didymos_x = []
didymos_y = []

wcs = WCS(headers[0])

for ra1,dec1 in zip(Didymos_ra1, Didymos_dec1):
    
    c = SkyCoord(ra1, dec1, frame='icrs', unit='deg')
    x, y = wcs.world_to_pixel(c)
    
    didymos_x.append(x)
    didymos_y.append(y)
    x = 0
    y = 0

### 7. Centroiding all Celestial Objects (Known Stars as well as Didymos)

In [ ]:
### Using Weighted Mean to Calculate Accurate Centroiding of Each Celestial Object ###

def find_complete_obj(image,x,y):

    centroided_coords = []
    
    #print(y)
    background = np.mean(image_data)
    
    
    for b,a in zip(x,y):
        a = int(a)
        b = int(b)
        n = 12
        square = image[a-n:a+n, b-n:b+n]
        
        weighted_x = []
        weighted_y = []
        
        for xc in range(len(square)):
            for yc in range(len(square[xc])):
                weighted_x.append((square[xc,yc]**1)*(a+xc))
                weighted_y.append((square[xc,yc]**1)*(b+yc))
                
        x_centroid = np.sum(weighted_x)/np.sum(square**1)
        y_centroid = np.sum(weighted_y)/np.sum(square**1)
        
        centroided_coords.append([y_centroid-n, x_centroid-n])
        
    return centroided_coords
        

In [ ]:
centroids = np.array(find_complete_obj(image_data, star_x,star_y))
centroids_didy = (np.array(find_complete_obj(images[i], didymos_x, didymos_y)))

In [ ]:
### Show Position of all Celestial Objects on Each Image ###

for i in range(len(images)):
    image_data = images[i]
    plt.figure(figsize=(15, 15))
    
    plt.imshow(image_data, cmap='gray', vmin=image_data.mean()-0.2*image_data.std(), vmax=image_data.mean()+0.2*image_data.std())

    
    plt.scatter(centroids_didy[i][0], centroids_didy[i][1], 10)
    plt.scatter(centroids[:,0], centroids[:,1], 10)

    plt.show()